In [1]:
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from datetime import datetime
from google.colab import drive
import os, zipfile, glob
import pickle

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Italian books from the gutenberg.org for text generation (top 5 in volume with the right encoding)

'65391-0'
'52377-0'
'48188-0'
'47278-0'
'56431-0'

In [19]:
!wget "https://www.gutenberg.org/files/65391/65391-0.txt"

--2022-01-18 18:37:05--  https://www.gutenberg.org/files/65391/65391-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2828283 (2.7M) [text/plain]
Saving to: ‘65391-0.txt.1’

65391-0.txt.1       100%[===================>]   2.70M  17.7MB/s    in 0.2s    

2022-01-18 18:37:26 (17.7 MB/s) - ‘65391-0.txt.1’ saved [2828283/2828283]



In [20]:
!wget "https://www.gutenberg.org/files/52377/52377-0.txt"

--2022-01-18 18:37:26--  https://www.gutenberg.org/files/52377/52377-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2570135 (2.5M) [text/plain]
Saving to: ‘52377-0.txt.1’

52377-0.txt.1       100%[===================>]   2.45M  16.1MB/s    in 0.2s    

2022-01-18 18:37:30 (16.1 MB/s) - ‘52377-0.txt.1’ saved [2570135/2570135]



In [21]:
!wget "https://www.gutenberg.org/files/48188/48188-0.txt"

--2022-01-18 18:37:30--  https://www.gutenberg.org/files/48188/48188-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2537076 (2.4M) [text/plain]
Saving to: ‘48188-0.txt.1’

48188-0.txt.1       100%[===================>]   2.42M  16.0MB/s    in 0.2s    

2022-01-18 18:37:39 (16.0 MB/s) - ‘48188-0.txt.1’ saved [2537076/2537076]



In [22]:
!wget "https://www.gutenberg.org/files/47278/47278-0.txt"

--2022-01-18 18:37:39--  https://www.gutenberg.org/files/47278/47278-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2521913 (2.4M) [text/plain]
Saving to: ‘47278-0.txt.1’

47278-0.txt.1       100%[===================>]   2.40M  14.9MB/s    in 0.2s    

2022-01-18 18:37:47 (14.9 MB/s) - ‘47278-0.txt.1’ saved [2521913/2521913]



In [23]:
!wget "https://www.gutenberg.org/files/56431/56431-0.txt"

--2022-01-18 18:37:47--  https://www.gutenberg.org/files/56431/56431-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2512215 (2.4M) [text/plain]
Saving to: ‘56431-0.txt.1’

56431-0.txt.1       100%[===================>]   2.40M  14.7MB/s    in 0.2s    

2022-01-18 18:37:53 (14.7 MB/s) - ‘56431-0.txt.1’ saved [2512215/2512215]



In [7]:
!pip install unidecode

In [8]:
import codecs
from bs4 import BeautifulSoup
import unidecode

In [24]:
list_books = ['65391-0', '52377-0', '48188-0', '47278-0', '56431-0']
text_add = ''
for b in list_books:
  with open(b + '.txt', 'r') as f:
    text = f.read()

    info_beg = text.find('Title: ')
    begin = text.find('*** START OF THIS PROJECT GUTENBERG EBOOK ')
    title = ''
    if begin > -1:
      for i in text[info_beg : begin].split('\n'):
        if i.find('Title: ') > -1:
          title = i.split('Title: ')[1]
          begin_2 = begin + 42 + len(title) + 3
                
    else:
      begin = text.find('START OF THE PROJECT GUTENBERG EBOOK ')
      for i in text[info_beg : begin].split('\n'):
        if i.find('Title: ') > -1:
          title = i.split('Title: ')[1]
          begin_2 = begin + 37 + len(title) + 3

    end = text.find('*** END OF THIS PROJECT GUTENBERG EBOOK')
    if end == -1:
      end = text.find('*** END')
    text = text[begin_2 : end]

    #remove html tags from text
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")
    
    #remove accented characters from text, e.g. café
    text = unidecode.unidecode(text)

    text_add = text_add + text + '\n'

In [25]:
len(text.split(' '))

356603

In [27]:
# Create two dictionaries:
# int2char -- maps integers to characters
# char2int -- maps characters to unique integers

int2char = dict(enumerate(set(text)))
char2int = { val: idx  for idx, val in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [28]:
len(char2int)

78

In [9]:
def one_hot_encode(arr, n_labels):
    
    # arr - array of integers
    # n_labels - number of labels (the size of a one-hot-encoded vector)

    one_hot = np.eye(n_labels)[arr]
    
    return one_hot

In [10]:
# check that the function works correctly
test_indx = np.array([[7, 2, 5]])
one_hot = one_hot_encode(test_indx, 8)

print(one_hot)

[[[0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]]]


In [20]:
def get_batches(arr, batch_size, seq_length):
    # Create a generator that returns batches of size batch_size x seq_length
    
    batch_size_total = batch_size * seq_length
    ## Get the number of batches we can make
    n_batches = int(len(arr) / batch_size_total)
    
    ## Keep only enough characters to make full batches
    arr = arr[: n_batches*batch_size_total] 
    
    ## Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1)) 
    
    ## Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n : n + seq_length ]
        # The target is a version of x shifted by one (do not forget border conditions)
        y = np.zeros_like(x)
        y[:, :-1] = x[:, 1:]
        if n + seq_length < arr.shape[1]:
            y[:, -1] = arr[:, n + seq_length]
        else:
            arr[:, 0]
        yield x, y

In [21]:
batches = get_batches(encoded, 4, 30)
x, y = next(batches)

In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU


In [12]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # Define the LSTM layer
        ## YOUR CODE HERE
        self.lstm = nn.LSTM(input_size=len(self.chars), hidden_size=self.n_hidden, 
                            num_layers=self.n_layers, dropout=self.drop_prob, 
                            batch_first=True)

        # Define a dropout layer
        ## YOUR CODE HERE
        self.dropout = nn.Dropout(self.drop_prob)

        # Define the final, fully-connected output layer
        ## YOUR CODE HERE
        self.linear = nn.Linear(in_features=self.n_hidden, out_features=len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        # Get the outputs and the new hidden state from the lstm
        ## YOUR CODE HERE
        out, hidden = self.lstm(x.float(), hidden)

        # Pass through a dropout layer
        ## YOUR CODE HERE
        out = self.dropout(out)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        ## YOUR CODE HERE
        out = out.reshape(out.size()[0] * out.size()[1], self.n_hidden)


        # Put x through the fully-connected layer
        ## YOUR CODE HERE
        out = self.linear(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden    

In [24]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net(inputs, h)
            
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                now = datetime.now()
                dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)),
                      "Time: ", dt_string)

In [25]:
n_hidden = 512 
n_layers = 2

chars = tuple(set(text))
net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(78, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=512, out_features=78, bias=True)
)


In [26]:
with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_it/dict_chars.pickle', 'wb') as f:
    pickle.dump(chars, f)

In [27]:
with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_it/dict_chars.pickle', 'rb') as f:
  c = pickle.load(f)

In [28]:
print(c)

('!', 'G', ',', 'H', 'o', 'm', '[', 'i', 'N', '-', '(', 'a', 'R', 'M', 'e', 'h', 'B', 'b', '\n', 'x', '?', '1', 'X', ';', 'V', '9', "'", 'r', '8', '4', 's', 'w', ':', 'v', 'j', 'O', 'c', '/', 'z', 't', 'L', 'l', 'd', 'F', 'U', 'J', 'S', 'g', 'P', '*', '0', ')', 'n', 'q', 'p', 'D', 'u', 'Q', 'I', 'Z', 'E', 'C', '_', '.', 'k', '6', ' ', '5', 'T', 'A', 'K', '3', 'y', ']', 'Y', 'f', '7', '2')


In [29]:
batch_size = 256
seq_length = 100 
n_epochs = 80

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=500)

Epoch: 6/80... Step: 500... Loss: 1.9112... Val Loss: 1.8159 Time:  18/01/2022 11:23:49
Epoch: 12/80... Step: 1000... Loss: 1.5220... Val Loss: 1.4529 Time:  18/01/2022 11:25:24
Epoch: 18/80... Step: 1500... Loss: 1.3952... Val Loss: 1.3166 Time:  18/01/2022 11:26:57
Epoch: 24/80... Step: 2000... Loss: 1.2989... Val Loss: 1.2521 Time:  18/01/2022 11:28:30
Epoch: 30/80... Step: 2500... Loss: 1.1976... Val Loss: 1.2138 Time:  18/01/2022 11:30:04
Epoch: 36/80... Step: 3000... Loss: 1.1694... Val Loss: 1.1918 Time:  18/01/2022 11:31:36
Epoch: 42/80... Step: 3500... Loss: 1.1567... Val Loss: 1.1766 Time:  18/01/2022 11:33:09
Epoch: 48/80... Step: 4000... Loss: 1.1256... Val Loss: 1.1676 Time:  18/01/2022 11:34:41
Epoch: 53/80... Step: 4500... Loss: 1.1147... Val Loss: 1.1641 Time:  18/01/2022 11:36:15
Epoch: 59/80... Step: 5000... Loss: 1.0794... Val Loss: 1.1628 Time:  18/01/2022 11:37:48
Epoch: 65/80... Step: 5500... Loss: 1.0668... Val Loss: 1.1712 Time:  18/01/2022 11:39:22
Epoch: 71/80

In [30]:
path = "/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_it/text_generator.pt" 
torch.save(net.state_dict(), path)


In [29]:
path = "/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_it/text_generator.pt" 


with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_it/dict_chars.pickle', 'rb') as f:
    chars_2 = pickle.load(f)

n_hidden = 512  
n_layers = 2
net_2 = CharRNN(chars_2, n_hidden, n_layers)
net_2.load_state_dict(torch.load(path))

<All keys matched successfully>

In [30]:
net_2.eval()

CharRNN(
  (lstm): LSTM(78, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=512, out_features=78, bias=True)
)

In [15]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [16]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [31]:
# Example of generated text
print(sample(net_2, 8000, prime='Ora passa il carattere precedente e prendine uno nuovo', top_k=20))

Ora passa il carattere precedente e prendine uno nuovo alla pace: il che resto condonuto la
sua consunta contra de' Veneziani, e volesse essere
in terra possona di quelle.
E ricupero _Giovanni Maria della
Navarro_, e di vigor della lega e dilotta tenuta de' Bisogna: santo.
Trovavasi a Firenze un principe, poco poita da _Filippo
III_ suo fratello _Carlo VIII_ re di Spagna, il ritirarsi soccorso di quelle nozze, e diedero
limesi di chiedere
un vicile ostia che notava ai capitani del papa rimadessero i suoi castidoli d'armata. Cola fu da meno o fra
gl'imperiali, ebbe in procrame alcuna di vederse non
poche nemiche sfagni delle lor due glorie attribuito a quella parte, convenne compartrone di pace. Negli Svizzeri era sicuro
dalle minaccie di lunghive,
e presi se non quasi tutti
ogni di fine.

Sorse la pupa il papa, a guerra agl'interessi, e ne i suoi Franzesi e dello stesso novello
concelto per la repubblica.
Passarono quei di esso Francesco s'oggide ricorsa quella del suo armamento, col m

In [32]:
text_ph = text.split('. ')

In [33]:
list_b1 = [ i.split(' ')[0].lower() for i in text_ph]
list_b2 = [ ' '.join(j.lower() for j in i.split(' ')[: 2]) for i in text_ph]
list_b3 = [ ' '.join(j.lower() for j in i.split(' ')[: 3]) for i in text_ph]
list_b4 = [ ' '.join(j.lower() for j in i.split(' ')[: 4]) for i in text_ph]

In [34]:
Counter(list_b1).most_common(10)

[('ma', 566),
 ('il', 283),
 ('non', 267),
 ('fu', 251),
 ('per', 230),
 ('in', 212),
 ('e', 210),
 ('si', 206),
 ('di', 172),
 ('nel', 161)]

In [35]:
import re

In [36]:
list_of_begins = set()
for i in Counter(list_b1).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

for i in Counter(list_b2).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

for i in Counter(list_b3).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

In [37]:
list(list_of_begins)

['Il',
 'In',
 'Ma',
 'Indiz',
 'Ma non',
 'Ma il',
 'Non si',
 '2rer',
 'Iitom',
 'Venet.tom',
 'Nel di',
 'Non',
 'Fu',
 'Ne si dee',
 'Si',
 'Di',
 'A questo',
 'Rer',
 'Per',
 'Ital',
 'E',
 'Nel']

In [38]:
import random

In [ ]:

num_docs = 15000

path = '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_it/'

for i in range(num_docs):
  start = random.choice(list(list_of_begins))
  text_gen = sample(net, 8000, prime= start, top_k=10)

  with open(path + 'doc_'+str(i)+'.txt', 'w+') as output_file:
    try:
      output_file.write(text_gen)
    except:
      continue
  if i % 200 == 0:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print(dt_string, '| ',  'number of processed files: ' + str(i), round(i/num_docs, 3)*100, '%')


18/01/2022 11:45:42 |  number of processed files: 0 0.0 %
18/01/2022 11:59:33 |  number of processed files: 200 1.3 %
18/01/2022 12:13:32 |  number of processed files: 400 2.7 %
18/01/2022 12:27:23 |  number of processed files: 600 4.0 %
18/01/2022 12:41:12 |  number of processed files: 800 5.3 %
18/01/2022 12:55:07 |  number of processed files: 1000 6.7 %
18/01/2022 13:09:01 |  number of processed files: 1200 8.0 %
18/01/2022 13:22:55 |  number of processed files: 1400 9.3 %
18/01/2022 13:36:54 |  number of processed files: 1600 10.7 %
18/01/2022 13:51:00 |  number of processed files: 1800 12.0 %
18/01/2022 14:05:00 |  number of processed files: 2000 13.3 %
18/01/2022 14:18:56 |  number of processed files: 2200 14.7 %
18/01/2022 14:32:53 |  number of processed files: 2400 16.0 %
18/01/2022 14:46:45 |  number of processed files: 2600 17.299999999999997 %
18/01/2022 15:00:36 |  number of processed files: 2800 18.7 %
18/01/2022 15:14:29 |  number of processed files: 3000 20.0 %
18/01/202

In [39]:
# If the generator does not work completely, you need to run this code (it will start from the last place)

# This is the code to generate (to determine how many have already been generated)

list_gen = glob.glob('/content/drive/MyDrive/2022-01-15_Course_project/gen_text_it/*.txt')
list_num = [ int(i.split('/')[-1].split('.txt')[0].split('doc_')[-1]) for i in list_gen]
max(list_num), min(list_num)


(5855, 0)

In [41]:
# If the generator does not work completely, you need to run this code (it will start from the last place)

num_docs = 15000

path = '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_it/'

for i in range(max(list_num), num_docs, 1):
  start = random.choice(list(list_of_begins))
  text_gen = sample(net_2, 8000, prime= start, top_k=10)

  with open(path + 'doc_'+str(i)+'.txt', 'w+') as output_file:
    try:
      output_file.write(text_gen)
    except:
      continue
  if i % 200 == 0:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print(dt_string, '| ',  'number of processed files: ' + str(i), round(i/num_docs, 3)*100, '%')

18/01/2022 18:50:38 |  number of processed files: 6000 40.0 %
18/01/2022 19:04:34 |  number of processed files: 6200 41.3 %
18/01/2022 19:18:30 |  number of processed files: 6400 42.699999999999996 %
18/01/2022 19:32:28 |  number of processed files: 6600 44.0 %
18/01/2022 19:46:28 |  number of processed files: 6800 45.300000000000004 %
18/01/2022 20:00:30 |  number of processed files: 7000 46.7 %
18/01/2022 20:14:37 |  number of processed files: 7200 48.0 %
18/01/2022 20:28:42 |  number of processed files: 7400 49.3 %
18/01/2022 20:42:43 |  number of processed files: 7600 50.7 %
18/01/2022 20:56:43 |  number of processed files: 7800 52.0 %
18/01/2022 21:10:41 |  number of processed files: 8000 53.300000000000004 %
18/01/2022 21:24:38 |  number of processed files: 8200 54.7 %
18/01/2022 21:38:34 |  number of processed files: 8400 56.00000000000001 %
18/01/2022 21:52:33 |  number of processed files: 8600 57.3 %
18/01/2022 22:06:32 |  number of processed files: 8800 58.699999999999996 %
1